In [ ]:
import pandas as pd
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
filedir = '<your folder here>'

In [ ]:
#df = pd.read_csv('data/landsat8_data_with_fire.csv')

df = pd.read_csv(os.path.join(filedir, 'dataset_path40_39_with_fire_v1.csv'))
                

df=df.drop(columns=["Unnamed: 0","Unnamed: 0.1"])
df.head(2)

In [ ]:
df_fire = df
df_fire['fire_nofire'] = df_fire['fire_nofire'].map(lambda x : 'no_fire' if x == 'no fire' else 'fire')
df_fire.head(2)
df = df_fire

In [ ]:
df_fire = df_fire[(df_fire.width == 512) & (df_fire.height == 512)]

In [ ]:
df_dataset = df_fire[df_fire.fire_nofire == 'fire'].append(df_fire[df_fire.fire_nofire == 'no_fire'][0:2000])

In [ ]:
df_dataset.shape

In [ ]:
from PIL import TiffImagePlugin

def resize(image, width, height):
    #print(image)
    
    #im1 = Image.open(image)
    
    width = width
    height = height
    # use one of these filter options to resize the image
    im2 = image.resize((width, height), Image.NEAREST)      # use nearest neighbour
    return im2

'''def resize(filename, width, height):
    #print(image)
    
    im1 = Image.open(filename)
    
    width = width
    height = height
    # use one of these filter options to resize the image
    im2 = im1.resize((width, height), Image.NEAREST)      # use nearest neighbour
    return im2'''

In [ ]:
def plotgrid(dataframe):
    images=[]
    classes=[]
    for item in df_date.itertuples():
        #print(item.filename)
        filename, ext = os.path.splitext(item.filename)
        rowcol = filename.split("_")
        loc = "{}:{}".format(rowcol[9], rowcol[10])
        filename = os.path.join(filedir, 'ndvijpegs',filename+".jpeg")
    
        #print(filename)
        im = Image.open(filename)
        
        im = resize(im, 100, 100)
        images.append(im)
        classes.append(item.fire_nofire)
    fig = plt.figure(figsize=(10, 10))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(16, 16),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
    #im_fire = Image.open("data/fire.png")
    #im_fire = resize(im_fire, 25,25)
    for ax, im, c in zip(grid, images,classes):
        # Iterating over the grid returns the Axes.
        ax.text(50,50, loc)
        ax.imshow(im,cmap='terrain')
        
        if c == 'fire':
            extent = 0, 100, 0, 100
            kwargs = {"alpha":0.1}
            rect = patches.Rectangle((0,0),100,100,linewidth=1,edgecolor='r',facecolor='r', **kwargs)
            #rect.set_alpha(0.1)
            ax.add_patch(rect)
            #ax.imshow(im_fire, alpha=0.1, extent=extent)
        




    plt.show()

In [ ]:

Path={}
for item in glob(os.path.join(filedir, 'ndvijpegs', '*.jpeg')):
    #print(item)
    filename = os.path.splitext(os.path.basename(item))[0]
    #print(filename)
    Path[filename+".TIF"] = item
 



In [ ]:
df_dataset['path'] = df_dataset['filename'].map(Path.get)
df_dataset.head(2)

In [ ]:
df_dataset['image'] = df_dataset['path'].map(lambda x: np.asarray(Image.open(x).resize((28,28))))

In [ ]:
df_dataset.head(2)


In [ ]:
df_dataset['image'] = df_dataset['image'].map(lambda x: np.array(x).reshape(x.shape[0],x.shape[1], 1))
#df_dataset['image'][0].reshape(512,512,1)

In [ ]:
#df_dataset = df_dataset[df_dataset.width == 512]

In [ ]:
X=df_dataset.drop(columns=["fire_nofire"])
Y=df_dataset["fire_nofire"]
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
     X, Y, test_size=0.33, random_state=42)

#X_train = np.array(X_train).reshape(10078, 28,28,1)


In [ ]:
x_train = np.asarray(X_train['image'].tolist())

x_test = np.asarray(X_test['image'].tolist())
#x_train = x_train.reshape(x_train.shape[0], *(512,512,1))
#x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))



In [ ]:
import keras

In [ ]:
#import sys
#!{sys.executable}  -m pip install --user keras

In [ ]:
#!{sys.executable}  -m pip install --user tensorflow

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

t1 = le.fit_transform(Y_train)
t2 = le.fit_transform(Y_test)
y_train = keras.utils.to_categorical(t1, num_classes=2)
y_test = keras.utils.to_categorical(t2, num_classes=2)
y_train.shape

In [ ]:
nsamples, nx, ny, nz = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))
d2_train_dataset.shape

In [ ]:
nsamples, nx, ny, nz = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))
d2_test_dataset.shape

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5)
clf.fit(d2_train_dataset, y_train)

In [ ]:
from sklearn.metrics import classification_report
y_pred = clf.predict(d2_test_dataset)
target_names=['fire', 'no_fire']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import roc_curve, auc
y_score = clf.predict(d2_test_dataset)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.exceptions import ConvergenceWarning

# different learning rate schedules and momentum parameters
params = [{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': 0,
           'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
           'nesterovs_momentum': False, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
           'nesterovs_momentum': True, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': 0,
           'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9,
           'nesterovs_momentum': True, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9,
           'nesterovs_momentum': False, 'learning_rate_init': 0.2},
          {'solver': 'adam', 'learning_rate_init': 0.01}]

labels = ["constant learning-rate", "constant with momentum",
          "constant with Nesterov's momentum",
          "inv-scaling learning-rate", "inv-scaling with momentum",
          "inv-scaling with Nesterov's momentum", "adam"]

plot_args = [{'c': 'red', 'linestyle': '-'},
             {'c': 'green', 'linestyle': '-'},
             {'c': 'blue', 'linestyle': '-'},
             {'c': 'red', 'linestyle': '--'},
             {'c': 'green', 'linestyle': '--'},
             {'c': 'blue', 'linestyle': '--'},
             {'c': 'black', 'linestyle': '-'}]


def plot_on_dataset(X, y, ax, name):
    # for each dataset, plot learning for each learning strategy
    print("\nlearning on dataset %s" % name)
    ax.set_title(name)

    X = MinMaxScaler().fit_transform(X)
    mlps = []
    if name == "digits":
        # digits is larger but converges fairly quickly
        max_iter = 15
    else:
        max_iter = 400

    for label, param in zip(labels, params):
        print("training: %s" % label)
        mlp = MLPClassifier(random_state=0,
                            max_iter=max_iter, **param)

        # some parameter combinations will not converge as can be seen on the
        # plots so they are ignored here
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning,
                                    module="sklearn")
            mlp.fit(X, y)

        mlps.append(mlp)
        print("Training set score: %f" % mlp.score(X, y))
        print("Training set loss: %f" % mlp.loss_)
    for mlp, label, args in zip(mlps, labels, plot_args):
        ax.plot(mlp.loss_curve_, label=label, **args)


fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# load / generate some toy datasets
#iris = datasets.load_iris()
#X_digits, y_digits = datasets.load_digits(return_X_y=True)
data_sets = [(d2_train_dataset, y_train)]

#for ax, data, name in zip(axes.ravel(), data_sets, ['Wildfire NDVI']):
plot_on_dataset(d2_train_dataset, y_train, ax=ax, name='Wildfire NDVI')

fig.legend(ax.get_lines(), labels, ncol=3, loc="upper center")
plt.show()


In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.patches as patches
def plotgrid1(dataframe):
    images=[]
    classes=[]
    predictions=[]
    for item in dataframe.itertuples():
        #print(item.filename)
        filename, ext = os.path.splitext(item.filename)
        rowcol = filename.split("_")
        loc = "{}:{}".format(rowcol[9], rowcol[10])
        filename = os.path.join(filedir, 'ndvijpegs',filename+".jpeg")
    
        #print(filename)
        im = Image.open(filename)
        #im = Image.fromarray(item.image)

        
        #im = resize(im, 100, 100)
        images.append(im)
        classes.append(item.actual)
        predictions.append(item.pred)
    fig = plt.figure(figsize=(10, 10))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(16, 16),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
    #im_fire = Image.open("data/fire.png")
    #im_fire = resize(im_fire, 25,25)
    for ax, im, c, p in zip(grid, images,classes, predictions):
        # Iterating over the grid returns the Axes.
        ax.text(50,50, loc)
        ax.imshow(im)
        
        if c == 'fire':
            #extent = 0, 100, 0, 100
            kwargs = {"alpha":0.1}
            rect = patches.Rectangle((0,0),50,50,linewidth=1,edgecolor='r',facecolor='r', **kwargs)
            #rect.set_alpha(0.1)
            ax.add_patch(rect)
            #ax.imshow(im_fire, alpha=0.1, extent=extent)
        if p == 'fire':
            #extent = 0, 100, 0, 100
            kwargs = {"alpha":0.5}
            rect = patches.Rectangle((50,50),100,100,linewidth=1,edgecolor='b',facecolor='b', **kwargs)
            #rect.set_alpha(0.1)
            ax.add_patch(rect)
            #ax.imshow(im_fire, alpha=0.1, extent=extent)
        




    plt.show()

plotgrid1(X_test)